# Geographical Aggregation (Tourism)

> Geographical Hierarchical Forecasting on Australian Tourism Data

In many applications, a set of time series is hierarchically organized. Examples include the presence of geographic levels, products, or categories that define different types of aggregations. In such scenarios, forecasters are often required to provide predictions for all disaggregate and aggregate series. A natural desire is for those predictions to be **"coherent"**, that is, for the bottom series to add up precisely to the forecasts of the aggregated series.

In this notebook we present an example on how to use `HierarchicalForecast` to produce coherent forecasts between geographical levels. We will use the classic Australian Domestic Tourism (`Tourism`) dataset, which contains monthly time series of the number of visitors to each state of Australia.

We will first load the `Tourism` data and produce base forecasts using an `AutoETS` model from `StatsForecast`, and then reconciliate the forecasts with several reconciliation algorithms from `HierarchicalForecast`. Finally, we show the performance is comparable with the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html) which uses the R package [fable](https://github.com/tidyverts/fable).

You can run these experiments using CPU or GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/AustralianDomesticTourism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install hierarchicalforecast statsforecast mlforecast datasetsforecast lightgbm sklearn neuralforecast

## 1. Load and Process Data

In this example we will use the [Tourism](https://otexts.com/fpp3/tourism.html) dataset from the [Forecasting: Principles and Practice](https://otexts.com/fpp3/) book.

The dataset only contains the time series at the lowest level, so we need to create the time series for all hierarchies.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
Y_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
Y_df = Y_df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
Y_df.insert(0, 'Country', 'Australia')
Y_df = Y_df[['Country', 'Region', 'State', 'Purpose', 'ds', 'y']]
Y_df['ds'] = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(Y_df["ds"], freq='Q').to_timestamp()
Y_df.head()

The dataset can be grouped in the following non-strictly hierarchical structure.

In [ ]:
spec = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]

Using the `aggregate` function from `HierarchicalForecast` we can get the full set of time series.

In [ ]:
from hierarchicalforecast.utils import aggregate

In [ ]:
%%capture
Y_df, S_df, tags = aggregate(Y_df, spec)

In [ ]:
Y_df.head()

In [ ]:
S_df.iloc[:5, :5]

In [ ]:
tags['Country/Purpose']

### Split Train/Test sets

We use the final two years (8 quarters) as test set.

In [ ]:
Y_test_df = Y_df.groupby('unique_id', as_index=False).tail(8)
Y_train_df = Y_df.drop(Y_test_df.index)

In [ ]:
Y_train_df.groupby('unique_id').size()

## 2. Computing base forecasts

The following cell computes the **base forecasts** for each time series in `Y_df` using the `ETS` model. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
%%capture
from statsforecast.models import AutoETS
from statsforecast.core import StatsForecast

In [ ]:
%%capture
fcst = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], 
                     freq='QS', n_jobs=-1)
Y_hat_df_stats = fcst.forecast(df=Y_train_df, h=8, fitted=True)
Y_fitted_df_stats = fcst.forecast_fitted_values()

### 2.1 Computing MLForecast models

In [ ]:
%%capture
import lightgbm as lgb
from sklearn.ensemble import HistGradientBoostingRegressor
from mlforecast.lag_transforms import ExpandingMean, RollingMean, ExpandingStd
from mlforecast.target_transforms import Differences
from mlforecast import MLForecast

In [ ]:
%%capture
mlf = MLForecast(
    models = {
        'lgbm': lgb.LGBMRegressor(verbosity=-1),
        'gbm':HistGradientBoostingRegressor()
    }, 
    freq='QS',
    target_transforms=[Differences([1, 4])],
    lags=[1, 2, 3, 4, 5, 6, 7, 8, 12],
    lag_transforms={  
        1: [ExpandingMean(), RollingMean(window_size=4)],
        4: [ExpandingMean(), RollingMean(window_size=2), RollingMean(window_size=4)],
        8: [RollingMean(window_size=4)]
    },
    date_features=['quarter', 'year']
)
mlf.fit(Y_train_df, fitted=True)
Y_hat_df_ml = mlf.predict(new_df=Y_train_df, h=8)
Y_fitted_df_ml = mlf.forecast_fitted_values()

### 2.2 Computing Neuralforecast models

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, MLP
from neuralforecast.losses.pytorch import MAE

In [ ]:
nf = NeuralForecast(
    models=[
        NBEATS(
            h=4,
            input_size=16,
            mlp_units=[[256, 256], [256, 256], [256, 256]],
            learning_rate=1e-3,
            loss=MAE(),
            random_seed=42
        ),
        MLP(
            h=4,
            input_size=16,
            num_layers=2,
            hidden_size=64,
            max_steps=500,
            learning_rate=1e-3,
            loss=MAE(),
            random_seed=42
        )
    ],
    freq='QS'
)
nf.fit(df=Y_train_df, val_size=4)
Y_hat_df_nf = nf.predict()
Y_fitted_df_nf = nf.predict_insample(step_size=4)

Seed set to 42
Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 615 K  | train
-------------------------------------------------------
614 K     Trainable params
180       Non-trainable params
615 K     Total params
2.461     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 71: 100%|██████████| 6/6 [00:00<00:00, 30.62it/s, v_num=862, train_loss_step=15.20, train_loss_epoch=21.10, valid_loss=30.40]  

`Trainer.fit` stopped: `max_steps=1000` reached.


Epoch 71: 100%|██████████| 6/6 [00:00<00:00, 30.40it/s, v_num=862, train_loss_step=15.20, train_loss_epoch=21.10, valid_loss=30.40]


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | mlp          | ModuleList    | 5.2 K  | train
4 | out          | Linear        | 260    | train
-------------------------------------------------------
5.5 K     Trainable params
0         Non-trainable params
5.5 K     Total params
0.022     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


...

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 342.48it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 268.01it/s]


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 379.27it/s]


In [ ]:
%%capture
Y_hat_df = Y_hat_df_stats.merge(Y_hat_df_ml, on=['unique_id', 'ds']).merge(Y_hat_df_nf, on=['unique_id', 'ds'])
Y_fitted_df = Y_fitted_df_stats.merge(Y_fitted_df_ml.drop(columns=['y']), on=['unique_id', 'ds']).merge(Y_fitted_df_nf.drop(columns=['cutoff', 'y']), on=['unique_id', 'ds'], how="left")

## 3. Reconcile forecasts

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. Since the hierarchy structure is not strict, we can't use methods such as `TopDown` or `MiddleOut`. In this example we use `BottomUp` and `MinTrace`.

In [ ]:
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation

In [ ]:
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S=S_df, tags=tags)

The dataframe `Y_rec_df` contains the reconciled forecasts.

In [ ]:
Y_rec_df.head()

,unique_id,ds,AutoETS,lgbm,knn,gbm,NBEATS,MLP,AutoETS/BottomUp,lgbm/BottomUp,...,knn/MinTrace_method-mint_shrink,gbm/MinTrace_method-mint_shrink,NBEATS/MinTrace_method-mint_shrink,MLP/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols,lgbm/MinTrace_method-ols,knn/MinTrace_method-ols,gbm/MinTrace_method-ols,NBEATS/MinTrace_method-ols,MLP/MinTrace_method-ols
0,Australia,2016-01-01,25990.068004,26523.543195,26500.318899,26720.376418,25906.072266,25596.566406,24381.672901,26945.553515,...,26709.734907,27004.711662,25459.239804,25247.044028,25894.419896,26641.438835,26563.968000,26792.200028,25711.648641,25496.079269
1,Australia,2016-04-01,24458.490282,24900.590405,24973.510958,24946.522366,24768.101562,23770.330078,22903.194016,26028.446233,...,25540.444227,25588.377171,24394.554295,23432.011005,24357.231461,25109.570985,25051.241552,25189.605116,24583.133324,23680.009340
2,Australia,2016-07-01,23974.055984,24139.018873,24438.339374,24554.762276,23682.156250,24544.498047,22411.401317,25345.135452,...,24971.413018,25303.378238,23378.419946,24427.526652,23865.928094,24451.714310,24562.536177,24822.730031,23575.178198,24530.602833
3,Australia,2016-10-01,24563.454495,26281.353485,26155.502665,26371.293563,25000.509766,26054.105469,23127.009693,26969.492032,...,26602.427914,26774.950824,24701.773136,25829.722533,24470.780870,26416.665730,26261.866112,26511.341898,24824.038550,26005.932850
4,Australia/ACT,2016-01-01,553.037571,611.947070,616.146669,611.156153,554.268188,582.645020,515.900790,621.156907,...,615.609803,617.307767,554.555923,582.762697,574.658651,591.749093,610.370245,581.131389,592.450086,599.256119


## 4. Evaluation 

The `HierarchicalForecast` package includes an `evaluate` function to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
from hierarchicalforecast.evaluation import evaluate
from utilsforecast.losses import rmse, mase
from functools import partial

In [ ]:
eval_tags = {}
eval_tags['Total'] = tags['Country']
eval_tags['Purpose'] = tags['Country/Purpose']
eval_tags['State'] = tags['Country/State']
eval_tags['Regions'] = tags['Country/State/Region']
eval_tags['Bottom'] = tags['Country/State/Region/Purpose']

df = Y_rec_df.merge(Y_test_df, on=['unique_id', 'ds'])
evaluation = evaluate(df = df,
                      tags = eval_tags,
                      train_df = Y_train_df,
                      metrics = [rmse,
                                 partial(mase, seasonality=4)])

#evaluation.columns = ['level', 'metric', 'Base', 'BottomUp', 'MinTrace(mint_shrink)', 'MinTrace(ols)']
#TODO: Format columns in a programatic level
numeric_cols = evaluation.select_dtypes(include="number").columns
evaluation[numeric_cols] = evaluation[numeric_cols].map('{:.2f}'.format).astype(np.float64)

### RMSE

The following table shows the performance measured using RMSE across levels for each reconciliation method.

In [ ]:
evaluation.query('metric == "rmse"')

,level,metric,AutoETS,lgbm,knn,gbm,NBEATS,MLP,AutoETS/BottomUp,lgbm/BottomUp,...,knn/MinTrace_method-mint_shrink,gbm/MinTrace_method-mint_shrink,NBEATS/MinTrace_method-mint_shrink,MLP/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols,lgbm/MinTrace_method-ols,knn/MinTrace_method-ols,gbm/MinTrace_method-ols,NBEATS/MinTrace_method-ols,MLP/MinTrace_method-ols
0,Total,rmse,963.09,318.13,386.53,378.75,848.03,634.04,2271.75,1099.85,...,750.33,899.50,1098.28,873.19,1027.21,433.82,430.58,561.48,960.51,698.94
2,Purpose,rmse,348.67,316.69,283.83,327.42,398.90,383.82,602.07,380.63,...,311.44,310.76,394.29,391.13,340.26,248.80,263.82,254.27,363.95,363.51
4,State,rmse,202.85,201.50,179.42,214.10,196.41,173.65,315.61,194.73,...,172.81,193.93,198.18,176.18,184.65,187.62,170.87,189.14,174.88,165.81
6,Regions,rmse,41.75,40.09,46.20,40.67,42.07,37.82,45.11,40.66,...,43.78,40.38,37.37,36.79,37.65,38.89,44.60,40.11,37.91,36.77
8,Bottom,rmse,16.73,18.19,21.56,18.18,18.35,16.44,16.73,18.19,...,20.89,18.03,16.17,16.07,15.85,18.22,21.74,18.27,16.44,16.12
10,Overall,rmse,31.14,29.96,33.14,30.58,32.55,29.08,40.00,32.57,...,33.22,31.16,30.58,29.34,29.37,29.07,32.70,29.75,29.75,28.43


### MASE


The following table shows the performance measured using MASE across levels for each reconciliation method.

In [ ]:
evaluation.query('metric == "mase"')

,level,metric,AutoETS,lgbm,knn,gbm,NBEATS,MLP,AutoETS/BottomUp,lgbm/BottomUp,...,knn/MinTrace_method-mint_shrink,gbm/MinTrace_method-mint_shrink,NBEATS/MinTrace_method-mint_shrink,MLP/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols,lgbm/MinTrace_method-ols,knn/MinTrace_method-ols,gbm/MinTrace_method-ols,NBEATS/MinTrace_method-ols,MLP/MinTrace_method-ols
1,Total,mase,0.78,0.24,0.35,0.30,0.85,0.61,2.38,1.04,...,0.64,0.88,1.04,0.83,0.84,0.32,0.36,0.50,0.93,0.67
3,Purpose,mase,0.78,0.92,0.74,0.94,0.93,0.97,1.72,1.33,...,0.99,1.05,0.99,1.06,0.74,0.68,0.71,0.74,0.80,0.92
5,State,mase,0.98,1.11,1.03,1.15,1.02,0.86,1.52,1.07,...,0.93,1.07,1.03,0.90,0.84,1.03,0.96,1.04,0.86,0.80
7,Regions,mase,0.96,0.99,1.13,1.01,1.03,0.91,1.01,1.01,...,1.08,0.98,0.93,0.89,0.87,0.96,1.11,0.99,0.93,0.88
9,Bottom,mase,0.88,1.08,1.31,1.11,0.95,0.87,0.88,1.08,...,1.27,1.09,0.87,0.86,0.91,1.11,1.42,1.16,0.93,0.88
11,Overall,mase,0.90,1.06,1.26,1.08,0.97,0.88,0.93,1.07,...,1.22,1.07,0.89,0.87,0.90,1.08,1.34,1.11,0.92,0.88


In [ ]:
# TODO: Put only the results that are in the table

### Comparison fable

Observe that we can recover the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html). The original results were calculated using the R package [fable](https://github.com/tidyverts/fable).

![Fable's reconciliation results](./imgs/AustralianDomesticTourism-results-fable.png)

### References
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)
- [Rob Hyndman, Alan Lee, Earo Wang, Shanika Wickramasuriya, and Maintainer Earo Wang (2021). "hts: Hierarchical and Grouped Time Series". URL https://CRAN.R-project.org/package=hts. R package version 0.3.1.](https://cran.r-project.org/web/packages/hts/index.html)
- [Mitchell O’Hara-Wild, Rob Hyndman, Earo Wang, Gabriel Caceres, Tim-Gunnar Hensel, and Timothy Hyndman (2021). "fable: Forecasting Models for Tidy Time Series". URL https://CRAN.R-project.org/package=fable. R package version 6.0.2.](https://CRAN.R-project.org/package=fable)